### Installing ultralytics to use YOLOv8

In [ ]:
from ultralytics import YOLO 
import cv2
import os

### Initiating a model as the yolo8x model

In [ ]:
model = YOLO('yolov8x')

### Running the predictions with the pre-trained model on 1 image

In [ ]:
# model.predict('data/1_train-val_1min_aalesund_from_start/img1/000001.jpg', save=True)

### Running the default preictor on the enire image set

In [ ]:
# model.predict('data/1_train-val_1min_aalesund_from_start/img1', save=True)

### Training a model to find the players

In [ ]:
options = {
    'data': 'yaml/players.yaml',
    'epochs': 50,
    'batch_size': 32,
    'img_size': 960,
    'project': 'players'
}

!yolo task=detect mode=train data={options['data']} epochs={options['epochs']} batch={options['batch_size']} imgsz={options['img_size']} 

### Training a model to find the ball

In [ ]:
model_ball = YOLO('yolo_weights/ball_weights.pt')

# Train the model directly specifying all the parameters
results_keypoints = model_ball.train(
    data='yaml/ball.yaml',
    epochs=50,
    batch=32,
    imgsz=1024,
    device='0',  
    workers=4,
    project='ball_training_best',
    exist_ok=True
)

### Training to find keypoints

In [ ]:
model_keypoints = YOLO('yolo_weights/keypoin_weights_new.pt')

# Train the model directly specifying all the parameters
results_keypoints = model_keypoints.train(
    data='yaml/keypoints.yaml',
    epochs=50,
    batch=30,
    imgsz=1024,
    device='0',  
    workers=4,
    exist_ok=True,
    project='keypoints_training'
)

#### Testing model to detect ball

In [ ]:
model_ball = YOLO('yolo_weights/new_best_ball?.pt')
image_folder = 'data/3_test_1min_hamkam_from_start/img1'
image_folder = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')])

In [ ]:
for image_path in image_folder:
    print(image_path)
    for result in model_ball.track(image_path, persist=True, stream=True, line_width=1, tracker='botsort.yaml'):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
    if key == 27:
        break
cv2.destroyAllWindows() 

#### Testing model to detect players (and ball)

In [ ]:
combined_model = YOLO('runs/detect/train3/weights/best.pt')
image_folder = 'data/3_test_1min_hamkam_from_start/img1'
image_folder = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')])


In [ ]:
for image_path in image_folder:
    print(image_path)
    for result in combined_model.track(image_path, persist=True, stream=True, line_width=1, tracker='botsort.yaml'):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
    if key == 27:
        break
cv2.destroyAllWindows()   

#### Testing model to detect keypoints

In [ ]:
model_keypoints = YOLO('yolo_weights/keypoint_test.pt')
image_folder = 'data/1_train-val_1min_aalesund_from_start/img1'
image_folder = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')])

In [ ]:
metrics = model_keypoints.val(data='yaml/keypoints.yaml', device='mps')

In [ ]:
display(metrics.box.map)

In [ ]:
image_path = 'data/3_test_1min_hamkam_from_start/img1/000436.jpg'
model_keypoints.predict(image_path, save=True, project='keypoints_test', conf=0.5)

In [ ]:
for image_path in image_folder:
    print(image_path)
    for result in model_keypoints.track(image_path, persist=True, stream=True, line_width=1, tracker='botsort.yaml', conf=0.5):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
    if key == 27:
        break
cv2.destroyAllWindows()   

# Evaluating the mapping

### Ball 

### Ball

In [2]:
model_ball = YOLO('yolo_weights/ball_weights.pt')
val = model_ball.val(data='yaml/test_ball.yaml')

Ultralytics YOLOv8.2.2 🚀 Python-3.10.13 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40338MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/testing/ball/labels.cache... 1412 images, 390 backgrounds, 0 corrupt: 100%|██████████| 1802/1802 [00:00<?, ?it/s]
/cluster/home/eirivarn/.conda/envs/tdt4265/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [01:26<00:00,  1.31it/s]


                   all       1802       1412      0.945      0.789        0.9       0.62
Speed: 0.7ms preprocess, 4.4ms inference, 0.0ms loss, 13.2ms postprocess per image
Results saved to runs/detect/val3


In [9]:
model_ball = YOLO('model_ball/ball_training_IDUN_E20_B20_img1024_it2/weights/best.pt')
val = model_ball.val(data='yaml/test_ball.yaml')

Ultralytics YOLOv8.2.2 🚀 Python-3.10.13 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40338MiB)
Model summary (fused): 268 layers, 68124531 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/testing/ball/labels.cache... 1412 images, 390 backgrounds, 0 corrupt: 100%|██████████| 1802/1802 [00:00<?, ?it/s]
/cluster/home/eirivarn/.conda/envs/tdt4265/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:43<00:00,  2.62it/s]


                   all       1802       1412      0.625      0.488      0.542      0.324
Speed: 0.2ms preprocess, 10.0ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val8


In [ ]:
model_ball = YOLO('model_ball/train_e100_imgsz1024_bz_12/weights/best.pt')
val = model_ball.val(data='yaml/test_ball.yaml')

### Players

In [ ]:
model_ball = YOLO('model_players/player_trainging_cybele_E50_B12_img1024_it1/weights/best.pt')
val = model_ball.val(data='yaml/test_players.yaml')

In [ ]:
model_ball = YOLO('yolo_weights/player_weights.pt')
val = model_ball.val(data='yaml/test_players.yaml')

### Keypoints

In [5]:
model_keypoints = YOLO('model_keypoints/train3/weights/best.pt')
val = model_keypoints.val(data='yaml/keypoints.yaml')

Ultralytics YOLOv8.2.2 🚀 Python-3.10.13 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40338MiB)
Model summary (fused): 268 layers, 68150532 parameters, 0 gradients, 257.5 GFLOPs


val: Scanning /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/labels.cache... 721 images, 0 backgrounds, 3 corrupt: 100%|██████████| 721/721 [00:00<?, ?it/s]

val: WARNING ⚠️ /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/images/000841.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0014]
val: WARNING ⚠️ /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/images/001188.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0007]
val: WARNING ⚠️ /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/images/001190.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0035]



/cluster/home/eirivarn/.conda/envs/tdt4265/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:23<00:00,  1.88it/s]


                   all        718       6763      0.994      0.997      0.995      0.955
                     0        718        219      0.995          1      0.995      0.972
                     1        718        663          1      0.999      0.995      0.968
                     2        718        193      0.998          1      0.995      0.934
                     3        718        364      0.996          1      0.995      0.969
                     4        718        289          1          1      0.995      0.959
                     5        718        178      0.999          1      0.995      0.966
                     6        718        101      0.998          1      0.995      0.974
                     7        718        161          1      0.991      0.995      0.946
                     8        718        177      0.999          1      0.995      0.958
                     9        718         98          1      0.982      0.995       0.95
                    1

In [7]:
model_keypoints = YOLO('model_keypoints/train3/weights/best.pt')
val = model_keypoints.val(data='yaml/keypoints.yaml')

Ultralytics YOLOv8.2.2 🚀 Python-3.10.13 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40338MiB)
Model summary (fused): 268 layers, 68150532 parameters, 0 gradients, 257.5 GFLOPs


val: Scanning /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/labels.cache... 721 images, 0 backgrounds, 3 corrupt: 100%|██████████| 721/721 [00:00<?, ?it/s]

val: WARNING ⚠️ /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/images/000841.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0014]
val: WARNING ⚠️ /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/images/001188.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0007]
val: WARNING ⚠️ /cluster/home/eirivarn/code/DeepLearningFotballAnalysis/data/yolo_data/combined_keypoints/validation/images/001190.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0035]



/cluster/home/eirivarn/.conda/envs/tdt4265/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:22<00:00,  2.02it/s]


                   all        718       6763      0.994      0.997      0.995      0.955
                     0        718        219      0.995          1      0.995      0.972
                     1        718        663          1      0.999      0.995      0.968
                     2        718        193      0.998          1      0.995      0.934
                     3        718        364      0.996          1      0.995      0.969
                     4        718        289          1          1      0.995      0.959
                     5        718        178      0.999          1      0.995      0.966
                     6        718        101      0.998          1      0.995      0.974
                     7        718        161          1      0.991      0.995      0.946
                     8        718        177      0.999          1      0.995      0.958
                     9        718         98          1      0.982      0.995       0.95
                    1